In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
#import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchvision.models import resnet34

In [2]:
#load the shadow model trained in the other python script
device = "cpu"
resnet_shadow = resnet34(pretrained = False,num_classes = 10).to(device) #resnet_target is the shadow model
resnet_cifar = torch.load("resnet34_shadow_cifar_overtrained2.pth",map_location=torch.device('cpu'))
resnet_shadow.load_state_dict(resnet_cifar)

/home/alex/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/alex/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [3]:
DATA_PATH = './resnet34/shadow.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)


#splitting
#only use train set here
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
dataloader = torch.utils.data.DataLoader(
    train_data, batch_size=
     1 , shuffle=False, num_workers=2)
testloader =  torch.utils.data.DataLoader(val_data, batch_size=1,
                                          shuffle=True, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)


In [4]:
#generate dataset for attack model
resnet_shadow.eval()
dataset_attack = []
with torch.no_grad():
    for images, labels in testloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,0])
        
with torch.no_grad():
    for images, labels in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)

        dataset_attack.append([top_values,1])
        



In [5]:

class Net_Attack(nn.Module):
    def __init__(self):
        super(Net_Attack, self).__init__()
        
        self.fc1 = nn.Linear(10, 256)
        self.batchnorm1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 1024)
        self.batchnorm2 = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(1024, 128)
        self.batchnorm3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x):

        x = F.relu(self.fc1(x))
        x =  self.batchnorm1(x)
        x = F.relu(self.fc2(x))
        x =  self.batchnorm2(x)
        x = F.relu(self.fc3(x))
        x =  self.batchnorm3(x)
        x = self.sigmoid(x)

        return x

In [6]:
print(dataset_attack[0])

[tensor([[13.7203,  1.3496,  1.0445]]), 0]


In [17]:


# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
standardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]


In [27]:
dataloader_attack = torch.utils.data.DataLoader(
    
    standardized_data_list, batch_size=32, shuffle=True, num_workers=2) #shuffled training data

In [28]:
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)  # Matches the output of fc1
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)  # Matches the output of fc2
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)  # Matches the output of fc3
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)  # Matches the output of fc4
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)   # Matches the output of fc5
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

# Ensure input dimension is correct
model = SimpleNN()
input_tensor = torch.randn(128, 3)  # Batch size of 10, each with 3 features
print(input_tensor.shape)
output = model(input_tensor)
print(output)

torch.Size([128, 3])
tensor([[0.5803],
        [0.4200],
        [0.4120],
        [0.3525],
        [0.6312],
        [0.6935],
        [0.4777],
        [0.4685],
        [0.4959],
        [0.2480],
        [0.4288],
        [0.4664],
        [0.2598],
        [0.6091],
        [0.4181],
        [0.4362],
        [0.2826],
        [0.4659],
        [0.5929],
        [0.3835],
        [0.4519],
        [0.4738],
        [0.4608],
        [0.4510],
        [0.4285],
        [0.4469],
        [0.5533],
        [0.3141],
        [0.3586],
        [0.4925],
        [0.4560],
        [0.3926],
        [0.4510],
        [0.5116],
        [0.7678],
        [0.3826],
        [0.4510],
        [0.3610],
        [0.4580],
        [0.4552],
        [0.3538],
        [0.5443],
        [0.4470],
        [0.4669],
        [0.4751],
        [0.5197],
        [0.4169],
        [0.3285],
        [0.2948],
        [0.3232],
        [0.4556],
        [0.5567],
        [0.2609],
        [0.2903],
       

In [29]:
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [30]:
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [31]:
# Define your training function

def train(model, dataloader, optimizer, criterion, epochs):
    for inputs, labels in dataloader:
        inputs = inputs.float()  # Ensures input tensors are floats
        labels = labels.float().view(-1, 1)  # Ensures labels are floats and reshaped correctly

        optimizer.zero_grad()
        outputs = model(inputs.squeeze(dim = 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

In [32]:
model.train()

SimpleNN(
  (fc1): Linear(in_features=3, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (bn4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=16, out_features=8, bias=True)
  (bn5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [33]:
#dict =  torch.load('attack_model.pth_3', map_location='cpu')
#model.load_state_dict(dict)

In [34]:
for inputs in dataloader_attack:
    inputs = inputs[0]
    print(inputs.shape)  # Check input shape consistenc
    print(inputs)
    print(inputs)
    outputs = model(inputs.squeeze(dim = 1))

torch.Size([32, 1, 3])
tensor([[[-1.5314, -0.1313, -0.4455]],

        [[-1.2928, -0.5020, -0.1125]],

        [[-0.8819,  0.6069,  1.8454]],

        [[-0.9210, -1.3684, -0.6179]],

        [[ 0.9892, -0.6659,  0.3058]],

        [[ 1.7645, -0.8388,  0.1477]],

        [[-0.5621,  1.2935,  0.7049]],

        [[ 0.1042,  1.5072, -1.1248]],

        [[ 0.8995,  0.1530, -1.4555]],

        [[-0.9767, -0.1802,  0.3822]],

        [[ 0.6027, -0.3629, -0.7264]],

        [[ 0.8945, -0.2218,  0.3416]],

        [[ 0.0958,  0.4125,  1.3817]],

        [[ 0.3946, -1.2838, -0.6687]],

        [[-0.3814, -0.0209, -0.4069]],

        [[ 0.7327,  0.2705, -0.5612]],

        [[ 1.7131,  0.9782,  0.5707]],

        [[-0.2684,  0.4525, -1.1921]],

        [[-0.3436,  1.0509,  0.2882]],

        [[-0.0806, -1.0279, -0.2397]],

        [[-0.0869,  0.9575, -0.7580]],

        [[ 0.9496, -0.1127, -1.0872]],

        [[ 0.0166, -0.9747, -0.0234]],

        [[-0.2641, -0.0823,  0.6742]],

        [[-0.2116

KeyboardInterrupt: 

In [70]:
model.train()
train(model,dataloader_attack,optimizer,criterion,250)

Loss: 0.5013183951377869
Loss: 0.6311392784118652
Loss: 0.5868628621101379
Loss: 0.6256136894226074
Loss: 0.5723690986633301
Loss: 0.739798903465271
Loss: 0.5945233702659607
Loss: 0.61891108751297
Loss: 0.5631478428840637
Loss: 0.5353920459747314
Loss: 0.630449116230011
Loss: 0.5764537453651428
Loss: 0.5169833898544312
Loss: 0.6571964621543884
Loss: 0.5783325433731079
Loss: 0.5884391665458679
Loss: 0.6197055578231812
Loss: 0.7522842288017273
Loss: 0.6175182461738586
Loss: 0.6747469902038574
Loss: 0.6305820345878601
Loss: 0.6783123016357422
Loss: 0.5146141648292542
Loss: 0.5238407850265503
Loss: 0.5886977910995483
Loss: 0.6398651599884033
Loss: 0.5694049596786499
Loss: 0.5716981887817383
Loss: 0.5456860065460205
Loss: 0.6455990076065063
Loss: 0.5047087669372559
Loss: 0.5094685554504395
Loss: 0.587340235710144
Loss: 0.6009442806243896
Loss: 0.5989408493041992
Loss: 0.5791116952896118
Loss: 0.6950997114181519
Loss: 0.633513331413269
Loss: 0.5376578569412231
Loss: 0.48962950706481934
Loss:

In [ ]:
torch.save(model.state_dict(), 'attack_model2.pth')

<h1>Evaluation</h1>

In [61]:
DATA_PATH = "eval.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
# Convert all tensors to the same dtype first

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1 , shuffle=False, num_workers=2)
#splitting
for batch_idx, (img, label, membership) in enumerate(dataloader):
    img = img.to(device)


# Define your evaluation function


In [62]:
dataset[0]

[tensor([[[0.3373, 0.3765, 0.3843,  ..., 0.2078, 0.2118, 0.2235],
          [0.4392, 0.5412, 0.5059,  ..., 0.2510, 0.2549, 0.2235],
          [0.4275, 0.4510, 0.4471,  ..., 0.2627, 0.2549, 0.2314],
          ...,
          [0.5373, 0.5647, 0.5529,  ..., 0.3490, 0.2745, 0.1765],
          [0.4157, 0.4353, 0.4549,  ..., 0.1608, 0.0941, 0.0745],
          [0.3882, 0.4000, 0.3961,  ..., 0.0627, 0.0627, 0.0627]],
 
         [[0.3765, 0.4314, 0.4314,  ..., 0.3412, 0.3333, 0.3490],
          [0.5529, 0.6784, 0.6275,  ..., 0.4039, 0.4157, 0.3804],
          [0.5059, 0.5569, 0.5765,  ..., 0.4235, 0.4118, 0.3843],
          ...,
          [0.6627, 0.7059, 0.7059,  ..., 0.3843, 0.2902, 0.1843],
          [0.4824, 0.5137, 0.5373,  ..., 0.1843, 0.1098, 0.0824],
          [0.4431, 0.4627, 0.4627,  ..., 0.0745, 0.0745, 0.0745]],
 
         [[0.1882, 0.2157, 0.2549,  ..., 0.4980, 0.4941, 0.5176],
          [0.1882, 0.2941, 0.2745,  ..., 0.5647, 0.5882, 0.5490],
          [0.3804, 0.2941, 0.2353,  ...,

In [63]:
dataset = []
resnet_shadow.eval()
with torch.no_grad():
    for images,_, b in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        print(logits)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        

        dataset.append([top_values,b])

tensor([[ 4.1837,  1.9899,  0.7940,  3.7374, -6.3575,  1.1705, -5.1319, -8.1269,
         -1.1223,  1.5514]])
tensor([[-6.5688, -4.4559,  2.0990,  8.6176, -2.2404, -0.1501, 10.1185, -2.3053,
         -7.3656, -1.8155]])
tensor([[-4.8385, 21.3618, -1.2031, -3.0123, -1.1403, -4.9268, -3.1162, -6.8571,
         -4.1152, -0.3516]])
tensor([[-9.6533, -8.8610, -0.7492,  7.8768, -0.2654, 19.4059, -2.9392,  2.3240,
         -6.7281, -6.4134]])
tensor([[ -1.5922,  14.4596,  -2.7383,  -4.9996,  -2.6969,  -4.2901,   2.3796,
         -10.0393,  -4.3558,   6.7165]])
tensor([[-3.6746e+00, -1.2006e+01,  2.8799e+00, -3.9514e-01, -1.7573e-03,
          6.5546e+00,  6.9025e+00,  1.8907e+00, -9.8879e+00,  1.1784e+00]])
tensor([[  3.8569,  -2.0829,  -0.4361,  -7.0202,  -2.5481,  -4.4735,  -2.7278,
         -10.7559,  15.3392,   4.3767]])
tensor([[-4.7745, -9.0919,  2.3283,  5.7900,  8.2868, -0.4958,  7.9966, -3.2547,
         -7.3594, -5.9545]])
tensor([[-4.7970, -1.1820,  2.7051,  2.2481, -5.4388, -1.558

In [57]:
resnet_target_model = resnet34(pretrained = False,num_classes = 10).to(device)
check = torch.load("models/resnet34_cifar10.pth", map_location=torch.device('cpu'))
resnet_target_model.load_state_dict(check["net"])
resnet_target_model.eval()

/home/alex/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/alex/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [76]:
#get posteriors from target
dataset_eval = []
with torch.no_grad():
    for images,_, member in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_target_model(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values #order poseri
        sorted_tensor, indices = torch.sort(top_values, dim=1,descending=True)
        dataset_eval.append([sorted_tensor, member.item()])
        


In [83]:
# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
dataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]

In [84]:
dataset_eval[0]

[tensor([[8.2969, 0.4971, 0.3079]]), 1]

In [85]:
dataloader_eval = torch.utils.data.DataLoader(
    
    dataset, batch_size=1 , shuffle=False, num_workers=2)

In [87]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for data in dataloader:
            inputs = data[0]
            
            labels = data[1].float()
            labels = torch.tensor(labels).float()
            #labels = torch.tensor(labels[0])
            optimizer.zero_grad()

            outputs = model(inputs.squeeze(dim=1))
            predicted = torch.round(outputs)  # Round the outputs to 0 or 1
            #predicted = abs(predicted -1)
            print(predicted,labels)

            total += labels.size(0)  # Increment the total count by batch size
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy

# Assuming you have a DataLoader called 'test_loader' for evaluation
# Create an instance of your model

# Assuming 'test_loader' is your DataLoader for evaluation
accuracy = evaluate(model, dataloader_eval)

print(f'Accuracy: {accuracy:.2f}')

/tmp/ipykernel_171584/100979714.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).float()


tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tenso

In [81]:
#1 - 0.26 = 0.74 > 0.68